A Python decorator function!

In Python, a decorator is a special type of function that can modify or extend the behavior of another function. A decorator is a function that takes another function as an argument and returns a new function that "wraps" the original function.

Here's a simple example of a decorator function:

In [2]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In this example, my_decorator is a function that takes func as an argument. It returns a new function wrapper, which calls func inside it. The @my_decorator syntax before say_hello is just a shortcut for say_hello = my_decorator(say_hello).

When you run say_hello(), it will print:

Something is happening before the function is called.
Hello!
Something is happening after the function is called.

Decorators are often used for things like logging, authentication, or caching.

Do you have a specific use case in mind for a decorator function, or would you like more examples?

https://habr.com/ru/companies/otus/articles/727590/

Следующая функция обладает этим свойством и может считаться самым простым декоратором, который только можно написать:

In [4]:
def null_decorator(func):
    return func

Как видите, null_decorator является вызываемым объектом, он принимает на вход другой вызываемый объект и возвращает тот же самый входной объект, не изменяя его.

Давайте используем его для декорирования (или обертывания) другой функции:

In [9]:
def greet():
    return 'Hello!'

greet = null_decorator(greet)

In [8]:
greet()

'Hello!'

В этом примере я определил функцию greet, а затем сразу же декорировал ее, прогнав ее через функцию null_decorator. Я знаю, пока это не выглядит чем-то очень полезным (мы ведь специально разработали декоратор null, чтобы он был бесполезным, верно?), но через некоторое время это прояснит, как работает синтаксис декораторов в Python.

Вместо того, чтобы явно вызывать null_decorator для greet, а затем переназначать переменную greet, можно использовать синтаксис Python @ для декорирования функции за один шаг:

In [10]:
@null_decorator
def greet():
    return 'Hello!'

Разместить строки @null_decorator перед определением функции — это то же самое, что сначала определить функцию, а затем применить к ней декоратор. Использование синтаксиса @ — это просто синтаксический сахар и сокращение для этого часто используемого шаблона.

Обратите внимание, что использование синтаксиса @ декорирует функцию непосредственно во время определения. Это затрудняет доступ к недекорированному оригиналу без хрупких хаков. Поэтому вы можете декорировать некоторые функции вручную, чтобы сохранить возможность вызова недекорированной функции.

**Декораторы могут изменять поведение**

Теперь, когда вы немного познакомились с синтаксисом декораторов, давайте напишем еще один декоратор, который действительно что-то делает и изменяет поведение декорируемой функции.

Вот немного более сложный декоратор, который преобразует результат декорированной функции в заглавные буквы:

In [11]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

Вместо того, чтобы просто возвращать входную функцию, как это делал декоратор null, декоратор uppercase определяет новую функцию на лету (замыкание) и использует ее для обертывания входной функции, чтобы изменить ее поведение во время вызова.

Замыкание wrapper имеет доступ к недекорированной входной функции и может свободно выполнять дополнительный код до и после вызова входной функции. (Технически, ей вообще не нужно вызывать входную функцию).

Обратите внимание, что до сих пор декорированная функция никогда не выполнялась. На самом деле вызов входной функции в этот момент не имеет никакого смысла — декоратор должен иметь возможность изменять поведение своей входной функции, когда она будет вызвана.

Пришло время увидеть декоратор uppercase в действии. Что произойдет, если декорировать им исходную функцию greet?

In [12]:
@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

Надеюсь, это был тот результат, которого вы ожидали. Давайте рассмотрим подробнее, что здесь произошло. В отличие от null_decorator, декоратор uppercase возвращает другой объект функции, когда он декорирует функцию:

In [13]:
greet

<function __main__.uppercase.<locals>.wrapper()>

In [14]:
null_decorator(greet)

<function __main__.uppercase.<locals>.wrapper()>

In [15]:
uppercase(greet)

<function __main__.uppercase.<locals>.wrapper()>

Как вы видели ранее, это необходимо для того, чтобы изменить поведение декорированной функции, когда она будет вызвана. Декоратор uppercase сам является функцией. И единственный способ повлиять на «будущее поведение» входной функции, которую он декорирует, — это заменить (или обернуть) входную функцию замыканием.

Вот почему uppercase определяет и возвращает другую функцию (замыкание), которую можно вызвать позднее, запустить исходную входную функцию и изменить ее результат.

Декораторы изменяют поведение вызываемого объекта с помощью обертки, поэтому вам не нужно постоянно изменять оригинал. Вызываемый объект не подвергается постоянным изменениям — его поведение меняется только при декорировании.

Это позволяет «присоединять» к существующим функциям и классам повторно используемые модули, такие как журналирование и другие инструменты. Именно это делает декораторы такой мощной функцией в Python, которая часто используется в стандартной библиотеке и в пакетах сторонних разработчиков.

**Применение нескольких декораторов к одной функции**

Возможно, неудивительно, что к функции можно применить более одного декоратора. Это накапливает их эффекты, и именно это делает декораторы настолько полезными, как повторно используемые модули.

Вот пример. Следующие два декоратора оборачивают выводимую строку декорируемой функции в HTML-теги. Посмотрев на то, как вложены теги, можно увидеть, в каком порядке Python применяет несколько декораторов:

In [16]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

Теперь давайте возьмем эти два декоратора и применим их к нашей функции greet одновременно. Для этого можно использовать обычный синтаксис @ и просто «уложить» несколько декораторов поверх одной функции:

In [17]:
@strong
@emphasis
def greet():
    return 'Hello!'

Какой результат вы ожидаете увидеть, если запустите декорированную функцию? Будет ли декоратор @emphasis первым добавлять свой тег <em> или @strong  имеет приоритет? Вот что происходит, когда вы вызываете декорированную функцию:

In [18]:
greet()

'<strong><em>Hello!</em></strong>'

Здесь хорошо видно, в каком порядке применялись декораторы: снизу вверх. Сначала входная функция была обернута декоратором @emphasis , а затем результирующая (декорированная) функция была снова обернута декоратором @strong.

Чтобы запомнить этот порядок снизу вверх, мне нравится называть такое поведение «стеком декораторов». Вы начинаете строить стек снизу, а затем продолжаете добавлять новые блоки сверху, чтобы проделать путь наверх.

Если разбить приведенный выше пример и не использовать синтаксис @ для применения декораторов, то цепочка вызовов функций декораторов выглядит следующим образом:

In [19]:
decorated_greet = strong(emphasis(greet))

Здесь снова видно, что сначала применяется декоратор emphasis, а затем полученная обернутая функция снова оборачивается декоратором strong.

Это также означает, что глубокие уровни наложения декораторов в конечном итоге будут влиять на производительность, поскольку они продолжают добавлять вызовы вложенных функций. Обычно это не является проблемой на практике, но об этом следует помнить, если вы работаете над кодом, требующим высокой производительности.

**Декорирование функций, принимающих аргументы**

Все примеры до сих пор декорировали только простую нульарную функцию greet, которая не принимала никаких аргументов. Поэтому декораторы, которые вы видели здесь до сих пор, не имели дела с передачей аргументов во входную функцию.

Если вы попытаетесь применить один из этих декораторов к функции, принимающей аргументы, он будет работать неправильно. Как декорировать функцию, принимающую произвольные аргументы?

Здесь на помощь приходит функция Python *args и **kwargs для работы с переменным количеством аргументов. Декоратор proxy использует эту возможность:

In [20]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

В этом декораторе есть два примечательных момента:

- Он использует операторы * и ** в определении замыкания wrapper для сбора всех позиционных и ключевых аргументов и хранения их в переменных (args и kwargs).

- Затем замыкание wrapper передает собранные аргументы исходной входной функции с помощью операторов «распаковки аргументов» * и **.

(Немного жаль, что значение операторов "звездочка" и "двойная звездочка" перегружено и меняется в зависимости от контекста, в котором они используются. Но я надеюсь, что вы поняли идею.)

Давайте расширим технику, заложенную в декораторе proxy, на более полезный практический пример. Вот декоратор trace, который выводит аргументы и результаты функции во время ее выполнения:

In [21]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
              f'with {args}, {kwargs}')

        original_result = func(*args, **kwargs)

        print(f'TRACE: {func.__name__}() '
              f'returned {original_result!r}')

        return original_result
    return wrapper

Декорировав функцию с помощью trace и вызвав ее, можно вывести аргументы, переданные декорированной функции, и ее возвращаемое значение. Это все еще в некоторой степени игрушечный пример, но в крайнем случае он станет отличным подспорьем для отладки:

In [22]:
@trace
def say(name, line):
    return f'{name}: {line}'

say('Jane', 'Hello, World')

TRACE: calling say() with ('Jane', 'Hello, World'), {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

Кстати, об отладке — есть несколько моментов, которые следует иметь в виду при отладке декораторов.

**Как писать «отлаживаемые» декораторы**

Когда вы используете декоратор, на самом деле вы заменяете одну функцию другой. Одним из недостатков этого процесса является то, что он «скрывает» некоторые метаданные, прикрепленные к исходной (недекорированной) функции.

Например, оригинальное имя функции, ее документационная строка (docstring) и список параметров скрываются замыканием:

In [23]:
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

decorated_greet = uppercase(greet)

Если вы попытаетесь получить доступ к любым метаданным этой функции, то вместо них вы увидите метаданные замыкания wrapper:

In [24]:
greet.__name__

'greet'

In [25]:
greet.__doc__

'Return a friendly greeting.'

In [26]:
decorated_greet.__name__

'wrapper'

In [27]:
decorated_greet.__doc__

Это делает отладку и работу с интерпретатором Python неудобной и сложной. К счастью, для этого есть быстрое решение: декоратор functools.wraps https://docs.python.org/3/library/functools.html#functools.wraps, включенный в стандартную библиотеку Python.

Вы можете использовать functools.wraps в своих собственных декораторах, чтобы скопировать потерянные метаданные из недекорированной функции в замыкание декоратора. Вот пример:

In [28]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

Применение functools.wraps к замыканию обертки, возвращаемому декоратором, переносит документационную строку и другие метаданные входной функции:

In [29]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

In [30]:
greet.__name__

'greet'

In [31]:
greet.__doc__

'Return a friendly greeting.'

Я бы рекомендовал использовать functools.wraps во всех декораторах, которые вы пишете сами. Это не займет много времени и избавит вас (и других) от головной боли при отладке в будущем.

Основные выводы
Декораторы определяют повторно используемые модули, которые можно применять к вызываемому объекту для изменения его поведения без постоянного изменения самого вызываемого объекта.

Синтаксис @ — это просто сокращение для вызова декоратора на входной функции. Несколько декораторов на одной функции применяются снизу вверх (наложение декораторов).

В качестве лучшей практики отладки используйте хелпер functools.wraps https://docs.python.org/3/library/functools.html#functools.wraps в своих декораторах, чтобы перенести метаданные из недекорированной вызываемой функции в декорированную

Статья про декораторы https://habr.com/ru/articles/750312/